#Регрессия

| Algorithm           | Sklearn                                    | Sklearn                                     | Custom                                      | Custom                                       |
|---------------------|--------------------------------------------|---------------------------------------------|---------------------------------------------|----------------------------------------------|
|                     | Baseline (MAE / RMSE / R²)                 | Improved (MAE / RMSE / R²)                  | Baseline (MAE / RMSE / R²)                  | Improved (MAE / RMSE / R²)                   |
| KNN                 | 116.36 / 479.45 / 0.7679                   | 143.39 / 285.05 / 0.7382                    | 154.29 / 521.70 / 0.7252                    | 96.50 / 227.01 / 0.8472                      |
| Linear Regression   | 215.55 / 605.22 / 0.6301                   | 115.99 / 229.30 / 0.8306                    | 215.55 / 605.22 / 0.6301                    | 115.99 / 229.30 / 0.8306                     |
| Decision Tree       | 137.02 / 562.10 / 0.6809                   | 89.24 / 204.12 / 0.8658                     | 137.02 / 562.10 / 0.6809                                            | 89.31 / 205.47 / 0.8640                      |
| Random Forest       | 91.97 / 300.37 / 0.9089                    | 77.00 / 186.33 / 0.8881                     | 92.11 / 295.91 / 0.9116                     | 88.73 / 192.36 / 0.8808                      |
| Gradient Boosting   | 107.89 / 300.97 / 0.9085                   | 83.21 / 179.16 / 0.8966                     | 108.86 / 302.92 / 0.9073                    | 90.23 / 187.27 / 0.8870                      |


##KNN

###Общие результаты.

Для алгоритма KNN на простом наборе признаков (TotalQuantity, NumLines, MeanPrice) использование исходных признаков без масштабирования приводит к R² ≈ 0.73 и относительно высоким значениям MAE и RMSE. Добавление стандартного масштабирования признаков заметно улучшает качество (R² возрастает до ~0.77), что отражает чувствительность KNN к масштабу признаков.
На улучшенном наборе признаков с расширенным описанием корзины, временными признаками и категорией страны попытка напрямую использовать высокоразмерный набор, включая qty_*, показала ухудшение качества. Даже после обрезки выбросов по целевой переменной улучшенный вариант KNN на «сложных» признаках не превзошёл аккуратно отмасштабированную базовую модель.
Кастомная реализация KNN дала результаты, сопоставимые с библиотечной версией: на простом датасете метрики совпадают вплоть до округления, а после обрезки выбросов и масштабирования на простом наборе признаков качество удалось улучшить до R² ≈ 0.85.

###Чувствительность и выводы.
KNN очень чувствителен к масштабу признаков и плохо переносит высокую размерность и разреженные one-hot признаки (qty_*). Для этого алгоритма наибольший эффект дали масштабирование и обрезка выбросов по TotalCheck. Добавление большого количества бинарных признаков состава корзины себя не оправдало. В целом KNN оказался пригоден в качестве опорного (baseline) алгоритма, однако по качеству уступил ансамблевым методам.

##Linear regression

###Общие результаты.
На простом наборе признаков линейная регрессия показала R² порядка 0.63, что подтверждает ограниченность простой линейной модели при описании суммарной стоимости чеков. Переход к улучшенному бейзлайну с использованием агрегированных признаков корзины, временных признаков, информации о составе корзины (qty_*) и обрезки выбросов по TotalCheck позволил повысить R² до ~0.83 и существенно снизить MAE и RMSE.
Регуляризация (Ridge, Lasso, ElasticNet) практически не изменила качество: улучшения по метрикам были на уровне сотых, что свидетельствует о том, что переобучение в данной постановке не является критической проблемой для линейной модели. Попытка использовать логарифмирование целевой переменной в данном контексте привела к значительному ухудшению метрик и не была принята.

###Чувствительность и выводы.
Линейная регрессия чувствительна к наличию выбросов по целевой переменной, поэтому обрезка верхнего хвоста распределения TotalCheck дала заметное улучшение. Ключевым фактором повышения качества стало расширение набора признаков и явное включение информации о составе корзины; масштабирование признаков и кодирование страны сыграли вспомогательную, но полезную роль. В сравнении с деревьями и ансамблями линейная регрессия остаётся хорошей базовой моделью, но не даёт максимального качества.

##Decision tree

###Общие результаты.
Базовая модель дерева решений на простом наборе признаков показала умеренное качество (R² ≈ 0.68), что ожидаемо для одиночного дерева с ограниченной глубиной и небольшим числом признаков. На улучшенном наборе признаков (агрегаты корзины, временные признаки, признаки qty_*, обрезка выбросов по целевой переменной) качество существенно выросло: R² достиг ~0.86.
Дополнительная настройка гиперпараметров (max_depth, min_samples_leaf, min_samples_split, max_features) через перебор по сетке позволила получить небольшое, но устойчивое улучшение (R² ≈ 0.866, уменьшение MAE и RMSE). Кастомная реализация дерева решений на тех же данных показала метрики, практически совпадающие с библиотечной, что подтверждает корректность реализации.

###Чувствительность и выводы.
Анализ важности признаков показал, что дерево решений очень сильно концентрируется на нескольких агрегированных характеристиках корзины, в первую очередь на TotalQuantity и MeanPrice. Такая концентрация объясняет как способность модели улавливать базовые зависимости, так и её склонность к переобучению: небольшие изменения в данных могут приводить к перестройке ключевых разбиений. В сравнении с Random Forest и Gradient Boosting одиночное дерево менее устойчиво и демонстрирует более скромные итоговые метрики, но при этом остаётся полезным ориентиром с точки зрения интерпретации и понимания структуры данных.

##Random forest

###Общие результаты.
Случайный лес на простом наборе признаков продемонстрировал значительное улучшение по сравнению с одиночным деревом и линейной регрессией. При более агрессивных настройках глубины дерева (до 16 уровней) и минимального числа объектов в листьях удалось получить R² порядка 0.91 при заметном снижении MAE и RMSE.
На улучшенном наборе признаков с обрезкой выбросов и масштабированием случайный лес показал ещё более высокое качество: MAE около 77 и RMSE около 186 при R² ~0.89, что делает его одной из лучших моделей в сравнении. Кастомная реализация случайного леса дала сопоставимое качество на простом датасете (R² чуть выше 0.91) и несколько более низкое качество на улучшенном наборе признаков, при существенно большем времени обучения.

###Чувствительность и выводы.
Анализ важности признаков показал, что лес опирается прежде всего на агрегированные характеристики корзины (TotalQuantity, MeanPrice, NumLines, NumUniqueItems), а временные признаки и отдельные qty_*-признаки играют вспомогательную роль. Это согласуется с предметной областью и результатами других моделей на основе деревьев.

Дополнительные эксперименты с гиперпараметрами (изменение глубины деревьев, минимального числа объектов в листьях и числа деревьев в ансамбле) показали, что случайный лес чувствителен к переобучению: слишком глубокие деревья улучшают R² на обучении, но не дают пропорционального выигрыша на тесте и могут приводить к росту MAE/RMSE. Тем не менее в подобранной конфигурации случайный лес демонстрирует хорошее соотношение качества и устойчивости, а также остаётся одной из сильных моделей среди протестированных алгоритмов.

##Gradient boosting

###Общие результаты.
На простом наборе признаков градиентный бустинг уже показал высокое качество (R² около 0.89–0.91 при разумных значениях MAE/RMSE). Добавление лёгкой настройки гиперпараметров (минимальное число объектов в листе/сплите) позволило немного улучшить баланс между MAE, RMSE и R².
На улучшенном наборе признаков с обрезкой выбросов, масштабированием числовых признаков и OHE для страны бустинг продемонстрировал один из лучших результатов: MAE ~83, RMSE ~179 и R² ~0.90. Кастомная реализация градиентного бустинга на тех же данных показала метрики, близкие к библиотечной реализации, но немного худшие, что объясняется отсутствием низкоуровневых оптимизаций и тонкой настройки.

###Чувствительность и выводы.
Анализ важности признаков показал, что модель опирается прежде всего на агрегированные характеристики корзины: TotalQuantity, MeanPrice и NumUniqueItems. Это соответствует предметной области: чем больше товаров и чем выше их средняя цена, тем выше ожидаемая сумма чека. Признаки конкретных популярных товаров (qty_*), а также временные и поведенческие признаки выступают в роли уточняющих факторов, позволяя модели корректировать предсказания в частных случаях, но не изменяя общую картину.

В совокупности результаты позволяют сделать вывод, что градиентный бустинг на улучшенном наборе признаков является одной из наиболее сильных моделей в задаче: он сочетает высокое качество предсказаний с разумной интерпретируемостью, а выявленная структура важности признаков подтверждает логичность и устойчивость построенной модели

##Total

В ходе экспериментов были последовательно рассмотрены пять алгоритмов регрессии: KNN, линейная регрессия, дерево решений, случайный лес и градиентный бустинг. Для каждого алгоритма сравнивались базовые модели на простом наборе признаков и улучшенные модели на расширенном, очищенном и предобработанном датасете, а также реализованы и протестированы собственные (кастомные) версии алгоритмов.
Результаты показывают, что:

KNN и линейная регрессия целесообразно использовать в качестве базовых ориентиров; они дают приемлемое качество после масштабирования и обрезки выбросов, но по метрикам заметно уступают ансамблевым методам на деревьях.

Дерево решений в одиночном варианте демонстрирует среднее качество, но при переходе к ансамблям (случайный лес, градиентный бустинг) качество существенно возрастает.

Наилучшие результаты по совокупности метрик (MAE, RMSE и R²) показали случайный лес и градиентный бустинг на улучшенном наборе признаков с предварительной очисткой данных и тщательной предобработкой.
Таким образом, для задачи прогнозирования суммы чека на основе данных о покупках наиболее рациональным выбором являются ансамблевые модели на деревьях решений, обученные на улучшенном бейзлайне с расширенными признаками и обработкой выбросов.

#Классификация